<img src="lilly_logo.png",width=200>
# <font color=#bb0000>__September 2018 - v1.0__</font>
## Supervised classification of SN tickets 

In [1]:
# Built-in python libraries
import sys
import os

#useful
import pprint
import pickle
from functools import reduce

# 3rd-party libraries 
import pandas as pd
from pandas import ExcelWriter
import numpy as np
    # plots
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns  

# time
from dateutil.parser import parse
from datetime import datetime

In [2]:
# Text Mining libs and packages
from nltk.tokenize import RegexpTokenizer
    # stopwords
from nltk.corpus import stopwords

# <font color=#003399>__Functions__</font>

In [3]:
def SubstractLists(list_one, list_two): 
    """
    SubstractLists(L1,L2) returns elements that belong to list L1 but not to L2.  
    """
    return sorted(list(set(list_one) - set(list_two))) 

# <font color=#003399>__0 - Data__</font>

A couple of sample strings from SN:

In [4]:
Res_one = "• [Résumé] Message d'interface en erreur sur PKG vers WES\n\n• [Conséquence factuelle] Impossible d'utiliser la palette.\nLes ES ne peuvent pas être utilisés sur la ligne.\n\n• [Résolution] Ouverture de INCTSK0057584 au business PKG.\nEn cours d'investigation\n\n• [QualityAssessment] Absence d'impact SISPQ à valider.\n[QAImpact] Pas d'impact qualité selon NADINE FAVROT le 31-MAY-2018"
Res_two ="• [Résumé] Message d'interface en erreur entre WES et PKG\n\n• [Conséquence factuelle] Impossible d'utiliser la palette.\nIdentification des articles impossible sur la ligne.\n\n• [Résolution] Ouverture de INCTSK0057582 au business PKG.\nEn cours d'investigation\n\nMessage retraité avec succès\nHP879844X HUMALOG MIX 50 KWPEN INDY SKPB A6 KS C903739A PRODUCTION PA06621000 460,000 ea Libéré Non Non 0000030379 B207R B207ST FEGERSHEIM Interne 01 040001405473 \n[QAImpact] Pas d'impact qualité. Vu avec Hélène LLORY le 08 JUIN 2018 \n\n• [QualityAssessment] Absence d'impact SISPQ à valider.\n[QAImpact] Pas d'impact qualité selon NADINE FAVROT le 31-MAY-2018"

Des_one="\r\nVoici la liste des messages d'interface en erreur répertoriés entre le 2018-5-30 à 07:00:00 et le 2018-5-31 à 07:00:00 :\r\n\r\n\r\n Message envoyé le 30/05/2018 07:49:34 ref_relation = WESSTOCKREMOVAL info = ERROR: Exception in com.pd.palcmd.PDPalConsumeInventoryCmd: Remaining quantity exceeds allowed deviation. \r\n\r\n==> Pas de traitement, PDC VIDE sur PKG\r\n=============================================================\r\n\r\n Message envoyé le 30/05/2018 08:42:19 ref_relation = WMMBID01_ZQM_BATCH01 info = Download of batch header C940834 failed. Item <CM0251> doesn't exist \r\n\r\n Message envoyé le 30/05/2018 08:52:19 ref_relation = WMMBID01_ZQM_BATCH01 info = Download of batch header C940836 failed. Item <CM0240> doesn't exist \r\n\r\n==> Pas de traitement \r\n=============================================================\r\n\r\n Message envoyé le 30/05/2018 08:53:35 ref_relation = WESSTOCKCREATION info = ERROR: Load carrier '0000030379' is already loaded in MES. \r\n\r\n==> INC5526371\r\n=============================================================\r\n\r\n Message envoyé le 30/05/2018 11:02:21 ref_relation = WMMBID01_ZQM_BATCH01 info = Download of batch header C940961 failed. Item <CM0717> doesn't exist \r\n\r\n Message envoyé le 30/05/2018 16:22:26 ref_relation = WMMBID01_ZQM_BATCH01 info = Download of batch header C901066 failed. Item <TBT705> doesn't exist \r\n\r\n==> Pas de traitement \r\n=============================================================\r\n\r\n Message envoyé le 31/05/2018 04:21:52 ref_relation = WESSTOCKREMOVAL info = ERROR: There are still TU's with quantity > 0 on load carrier 0000032361 \r\n\r\n==> Palette en B207R controlé\r\n=============================================================\r\n\r\n Message envoyé le 31/05/2018 06:13:54 ref_relation = WESSTOCKREMOVAL info = ERROR: There are still TU's with quantity > 0 on load carrier 0000015862 \r\n\r\n==> INC5526398\r\n=============================================================\r\n\r\n-- Fin --\r\n"

# <font color=#003399>__1 - Generate clean tokens__</font>

Define regex to tokenize

In [5]:
tokenizer = RegexpTokenizer("\W'+|\$[\d\.]+|\S+")

It looks fine on our samples (esp. time, date, and ')

In [6]:
tokens = tokenizer.tokenize(Des_one) ; tokens

['Voici',
 'la',
 'liste',
 'des',
 'messages',
 "d'interface",
 'en',
 'erreur',
 'répertoriés',
 'entre',
 'le',
 '2018-5-30',
 'à',
 '07:00:00',
 'et',
 'le',
 '2018-5-31',
 'à',
 '07:00:00',
 ':',
 'Message',
 'envoyé',
 'le',
 '30/05/2018',
 '07:49:34',
 'ref_relation',
 '=',
 'WESSTOCKREMOVAL',
 'info',
 '=',
 'ERROR:',
 'Exception',
 'in',
 'com.pd.palcmd.PDPalConsumeInventoryCmd:',
 'Remaining',
 'quantity',
 'exceeds',
 'allowed',
 'deviation.',
 '==>',
 'Pas',
 'de',
 'traitement,',
 'PDC',
 'VIDE',
 'sur',
 'PKG',
 '=============================================================',
 'Message',
 'envoyé',
 'le',
 '30/05/2018',
 '08:42:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'of',
 'batch',
 'header',
 'C940834',
 'failed.',
 'Item',
 '<CM0251>',
 "doesn't",
 'exist',
 'Message',
 'envoyé',
 'le',
 '30/05/2018',
 '08:52:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'of',
 'batch',
 'header',
 'C940836',

In [7]:
len(tokens)

219

### <font color=#ff6600>__Stopwords__</font>

Approach: 
    - Apply filters one by one (see filters below).
    - After applying each filter, generate a new sublist of tokens.
    - Compare new and old tokens and keep track of the removed list. 
 
List of filters:
    - F1: French stopwords.
    - F2: English stopwords.
    - F3: extra chars to get rid of.
    - F4: manually added extra stopwords.

**Filter 1**: NLTK French stop words

In [8]:
french_stops = set(stopwords.words('french'))

In [9]:
tokens_F1_FR=[word for word in tokens if word not in french_stops]; tokens_F1_FR

['Voici',
 'liste',
 'messages',
 "d'interface",
 'erreur',
 'répertoriés',
 'entre',
 '2018-5-30',
 '07:00:00',
 '2018-5-31',
 '07:00:00',
 ':',
 'Message',
 'envoyé',
 '30/05/2018',
 '07:49:34',
 'ref_relation',
 '=',
 'WESSTOCKREMOVAL',
 'info',
 '=',
 'ERROR:',
 'Exception',
 'in',
 'com.pd.palcmd.PDPalConsumeInventoryCmd:',
 'Remaining',
 'quantity',
 'exceeds',
 'allowed',
 'deviation.',
 '==>',
 'Pas',
 'traitement,',
 'PDC',
 'VIDE',
 'PKG',
 '=============================================================',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:42:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'of',
 'batch',
 'header',
 'C940834',
 'failed.',
 'Item',
 '<CM0251>',
 "doesn't",
 'exist',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:52:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'of',
 'batch',
 'header',
 'C940836',
 'failed.',
 'Item',
 '<CM0240>',
 "doesn't",
 'exist',
 '==>',
 'Pas',
 'traitement',
 '=

In [10]:
len(tokens_F1_FR)

196

Check removed tokens:

In [11]:
SubstractLists(tokens, tokens_F1_FR)

['de', 'des', 'en', 'et', 'la', 'le', 'on', 'sur', 'à']

**Filter 2**: NLTK English stop words

In [12]:
english_stops = set(stopwords.words('english'))

In [13]:
tokens_F2_EN=[word for word in tokens_F1_FR if word not in english_stops]; tokens_F2_EN

['Voici',
 'liste',
 'messages',
 "d'interface",
 'erreur',
 'répertoriés',
 'entre',
 '2018-5-30',
 '07:00:00',
 '2018-5-31',
 '07:00:00',
 ':',
 'Message',
 'envoyé',
 '30/05/2018',
 '07:49:34',
 'ref_relation',
 '=',
 'WESSTOCKREMOVAL',
 'info',
 '=',
 'ERROR:',
 'Exception',
 'com.pd.palcmd.PDPalConsumeInventoryCmd:',
 'Remaining',
 'quantity',
 'exceeds',
 'allowed',
 'deviation.',
 '==>',
 'Pas',
 'traitement,',
 'PDC',
 'VIDE',
 'PKG',
 '=============================================================',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:42:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'batch',
 'header',
 'C940834',
 'failed.',
 'Item',
 '<CM0251>',
 'exist',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:52:19',
 'ref_relation',
 '=',
 'WMMBID01_ZQM_BATCH01',
 'info',
 '=',
 'Download',
 'batch',
 'header',
 'C940836',
 'failed.',
 'Item',
 '<CM0240>',
 'exist',
 '==>',
 'Pas',
 'traitement',
 '==============================================

In [14]:
len(tokens_F2_EN)

181

Check removed tokens

In [15]:
SubstractLists(tokens_F1_FR, tokens_F2_EN)

['are', "doesn't", 'in', 'is', 'of', 'with']

**Filter 3**: extra chars to get rid of.

In [16]:
char_stops = [':',
              '=',
              '==>', 
              '=============================================================', 
              '>',
              '--']

In [17]:
tokens_F3_chars=[word for word in tokens_F2_EN if word not in char_stops]; tokens_F3_chars

['Voici',
 'liste',
 'messages',
 "d'interface",
 'erreur',
 'répertoriés',
 'entre',
 '2018-5-30',
 '07:00:00',
 '2018-5-31',
 '07:00:00',
 'Message',
 'envoyé',
 '30/05/2018',
 '07:49:34',
 'ref_relation',
 'WESSTOCKREMOVAL',
 'info',
 'ERROR:',
 'Exception',
 'com.pd.palcmd.PDPalConsumeInventoryCmd:',
 'Remaining',
 'quantity',
 'exceeds',
 'allowed',
 'deviation.',
 'Pas',
 'traitement,',
 'PDC',
 'VIDE',
 'PKG',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:42:19',
 'ref_relation',
 'WMMBID01_ZQM_BATCH01',
 'info',
 'Download',
 'batch',
 'header',
 'C940834',
 'failed.',
 'Item',
 '<CM0251>',
 'exist',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:52:19',
 'ref_relation',
 'WMMBID01_ZQM_BATCH01',
 'info',
 'Download',
 'batch',
 'header',
 'C940836',
 'failed.',
 'Item',
 '<CM0240>',
 'exist',
 'Pas',
 'traitement',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:53:35',
 'ref_relation',
 'WESSTOCKCREATION',
 'info',
 'ERROR:',
 'Load',
 'carrier',
 " '",
 "0000030379'",
 'already',
 'loade

In [18]:
len(tokens_F3_chars)

148

**Filter 4**: Manual extra stop words

In [19]:
manual_stops = ['Voici']

In [20]:
tokens_F4_manual=[word for word in tokens_F3_chars if word not in manual_stops]; tokens_F4_manual

['liste',
 'messages',
 "d'interface",
 'erreur',
 'répertoriés',
 'entre',
 '2018-5-30',
 '07:00:00',
 '2018-5-31',
 '07:00:00',
 'Message',
 'envoyé',
 '30/05/2018',
 '07:49:34',
 'ref_relation',
 'WESSTOCKREMOVAL',
 'info',
 'ERROR:',
 'Exception',
 'com.pd.palcmd.PDPalConsumeInventoryCmd:',
 'Remaining',
 'quantity',
 'exceeds',
 'allowed',
 'deviation.',
 'Pas',
 'traitement,',
 'PDC',
 'VIDE',
 'PKG',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:42:19',
 'ref_relation',
 'WMMBID01_ZQM_BATCH01',
 'info',
 'Download',
 'batch',
 'header',
 'C940834',
 'failed.',
 'Item',
 '<CM0251>',
 'exist',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:52:19',
 'ref_relation',
 'WMMBID01_ZQM_BATCH01',
 'info',
 'Download',
 'batch',
 'header',
 'C940836',
 'failed.',
 'Item',
 '<CM0240>',
 'exist',
 'Pas',
 'traitement',
 'Message',
 'envoyé',
 '30/05/2018',
 '08:53:35',
 'ref_relation',
 'WESSTOCKCREATION',
 'info',
 'ERROR:',
 'Load',
 'carrier',
 " '",
 "0000030379'",
 'already',
 'loaded',
 'MES.

In [21]:
len(tokens_F4_manual)

147

<font color=##4dd7ff>__ ======================__</font>
## <font color=##4dd7ff>__FORK ALERT!__</font>
In v1.0 we continue to get stats quickly. However here are some leads to follow later:
    - Stemming vs lemmetization. 
    Here is an ex. of comparison of different stemmers: 
        https://medium.com/greyatom/a-dive-into-natural-language-processing-103ae9b0a588
    - POS tags: should be useful of put together all date and times instead of have different tokens popping.     

# <font color=#003399>__2 - Frequency__</font>

N-grams

In [37]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures